<a href="https://colab.research.google.com/github/ronniewillaert/Biofabrication-Exercises/blob/main/Chapter3_Python_Exercises/Chapter3_Python_Exercise4_Perfusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 3 - Exercise 4: Organ-on-Chip Perfusion Optimizer
## Biofabrication Course - VU Brussels

This notebook covers:
- Cell metabolic requirements (oxygen, glucose)
- Perfusion rate calculations
- Nutrient delivery vs shear stress balance
- Cell viability predictions
- Long-term culture system design

KEY LEARNING FEATURES:
- Calculate optimal perfusion rates
- Balance nutrient delivery with shear stress
- Model oxygen and glucose gradients
- Design for different cell types
- Optimize for long-term culture

In [ ]:
# Chapter 3 - Exercise 4: Organ-on-Chip Perfusion Optimizer
# Biofabrication Course - VU Brussels
# Interactive Python Exercise

"""
WHAT'S SPECIAL ABOUT EXERCISE 4:

This notebook covers:
• Perfusion system design principles for maintaining cell viability in organ chips
• Nutrient delivery and waste removal calculations based on cell metabolic rates
• Residence time analysis to ensure adequate molecular exchange
• Oxygen and glucose consumption modeling for different cell types and densities
• Shear stress balance between physiological relevance and cell damage
• Pump selection and flow rate optimization for long-term culture
• Integration of sensors for real-time monitoring of chip conditions

KEY LEARNING FEATURES:
✓ Calculate optimal perfusion rates for different organ chips
✓ Balance nutrient delivery with shear stress constraints
✓ Model oxygen and glucose gradients in perfused channels
✓ Design perfusion systems for hepatocytes, endothelial cells, and other types
✓ Predict cell viability under different flow conditions
✓ Optimize for long-term culture (days to weeks)
✓ Compare static vs perfused culture outcomes
"""

import sys
!{sys.executable} -m pip install matplotlib numpy pandas seaborn plotly -q

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from matplotlib.patches import Rectangle, FancyBboxPatch, Circle, FancyArrowPatch
import plotly.graph_objects as go

print("="*80)
print("   CHAPTER 3 - EXERCISE 4: ORGAN-ON-CHIP PERFUSION OPTIMIZER")
print("="*80)
print()

# ============================================================================
# SECTION 1: CELL METABOLIC REQUIREMENTS
# ============================================================================

print("\n" + "="*80)
print("SECTION 1: CELL METABOLIC REQUIREMENTS")
print("="*80)

print("""
🧬 KEY METABOLIC PARAMETERS:

Oxygen consumption rate (OCR):
   • Typical range: 50-500 fmol/cell/s
   • Highly metabolic (hepatocytes): ~200 fmol/cell/s
   • Endothelial cells: ~100 fmol/cell/s
   • Epithelial cells: ~80 fmol/cell/s

Glucose consumption:
   • Range: 10-100 fmol/cell/s
   • ATP production: 30-36 ATP per glucose

Critical thresholds:
   • Oxygen: >20% of atmospheric (>40 μM) for viability
   • Glucose: >1 mM for most cell types
""")

# Cell type metabolic database
cell_metabolic_data = {
    'Hepatocytes': {
        'OCR': 200,  # fmol O2/cell/s
        'glucose_rate': 50,  # fmol glucose/cell/s
        'lactate_production': 80,  # fmol lactate/cell/s
        'shear_tolerance': 5,  # Pa
        'density': 5e5,  # cells/cm²
        'color': '#e74c3c'
    },
    'Endothelial cells': {
        'OCR': 100,
        'glucose_rate': 30,
        'lactate_production': 50,
        'shear_tolerance': 4,
        'density': 3e5,
        'color': '#3498db'
    },
    'Cardiomyocytes': {
        'OCR': 300,
        'glucose_rate': 70,
        'lactate_production': 100,
        'shear_tolerance': 2,
        'density': 4e5,
        'color': '#e67e22'
    },
    'Kidney cells': {
        'OCR': 150,
        'glucose_rate': 40,
        'lactate_production': 60,
        'shear_tolerance': 3,
        'density': 4e5,
        'color': '#9b59b6'
    },
    'Lung epithelial': {
        'OCR': 80,
        'glucose_rate': 25,
        'lactate_production': 40,
        'shear_tolerance': 1,
        'density': 2e5,
        'color': '#1abc9c'
    }
}

print("\n📊 CELL TYPE METABOLIC PROFILES:")
print("-" * 80)
print(f"{'Cell Type':<20} {'OCR':<15} {'Glucose':<15} {'Shear Tol.':<15} {'Density'}")
print(f"{'':20} {'(fmol/cell/s)':<15} {'(fmol/cell/s)':<15} {'(Pa)':<15} {'(cells/cm²)'}")
print("-" * 80)

for cell_type, data in cell_metabolic_data.items():
    print(f"{cell_type:<20} {data['OCR']:<15} {data['glucose_rate']:<15} "
          f"{data['shear_tolerance']:<15} {data['density']:.1e}")

# ============================================================================
# SECTION 2: PERFUSION SYSTEM CALCULATOR
# ============================================================================

print("\n" + "="*80)
print("SECTION 2: DESIGN YOUR PERFUSION SYSTEM")
print("="*80)

# Design parameters (STUDENTS CAN MODIFY THESE)
cell_type = 'Hepatocytes'  # MODIFY THIS: Choose from cell_metabolic_data keys
chip_area = 1.0  # cm² (MODIFY THIS: 0.1-10 cm²)
channel_height = 100e-4  # cm (100 μm) (MODIFY THIS)
flow_rate = 50  # μL/min (MODIFY THIS: 1-500)

print("\n🔬 YOUR DESIGN PARAMETERS:")
print("-" * 50)
print(f"Cell type:            {cell_type}")
print(f"Chip culture area:    {chip_area} cm²")
print(f"Channel height:       {channel_height*1e4:.0f} μm")
print(f"Flow rate:            {flow_rate} μL/min")

# Get cell properties
cell_props = cell_metabolic_data[cell_type]
cell_density = cell_props['density']
total_cells = chip_area * cell_density

print(f"\nCell density:         {cell_density:.1e} cells/cm²")
print(f"Total cells in chip:  {total_cells:.1e} cells")

# ============================================================================
# SECTION 3: NUTRIENT DELIVERY ANALYSIS
# ============================================================================

print("\n" + "="*80)
print("SECTION 3: NUTRIENT DELIVERY CALCULATIONS")
print("="*80)

def calculate_oxygen_consumption(total_cells, OCR):
    """Calculate total oxygen consumption rate"""
    # OCR in fmol/cell/s, convert to μmol/min
    consumption = (total_cells * OCR * 1e-15 * 60) / 1e-6  # μmol/min
    return consumption

def calculate_residence_time(chip_volume, flow_rate):
    """Calculate how long medium stays in chip"""
    # chip_volume in μL, flow_rate in μL/min
    residence = chip_volume / flow_rate  # minutes
    return residence

def calculate_oxygen_depletion(O2_consumption, flow_rate, inlet_O2=200):
    """Calculate oxygen depletion"""
    # inlet_O2 in μM (atmospheric ~200 μM)
    # O2_consumption in μmol/min
    # flow_rate in μL/min = mL/min * 1e-3

    flow_rate_L = flow_rate * 1e-6  # Convert to L/min
    depletion = O2_consumption / flow_rate_L  # μM
    outlet_O2 = inlet_O2 - depletion

    return outlet_O2, depletion

# Calculate for current design
chip_volume = chip_area * channel_height * 1000  # Convert to μL
O2_consumption = calculate_oxygen_consumption(total_cells, cell_props['OCR'])
glucose_consumption = calculate_oxygen_consumption(total_cells, cell_props['glucose_rate'])
residence = calculate_residence_time(chip_volume, flow_rate)

# Oxygen analysis
inlet_O2 = 200  # μM (atmospheric saturation)
outlet_O2, O2_depletion = calculate_oxygen_depletion(O2_consumption, flow_rate, inlet_O2)

# Glucose analysis (assuming 5 mM inlet)
inlet_glucose = 5000  # μM
flow_rate_L = flow_rate * 1e-6
glucose_depletion = glucose_consumption / flow_rate_L
outlet_glucose = inlet_glucose - glucose_depletion

print(f"\n💧 FLOW CHARACTERISTICS:")
print("-" * 50)
print(f"Chip volume:          {chip_volume:.2f} μL")
print(f"Residence time:       {residence:.2f} minutes")
print(f"Medium exchanges:     {60/residence:.1f} per hour")

print(f"\n🔬 METABOLITE ANALYSIS:")
print("-" * 50)
print(f"Total O2 consumption: {O2_consumption:.3f} μmol/min")
print(f"Inlet O2:             {inlet_O2:.1f} μM")
print(f"Outlet O2:            {outlet_O2:.1f} μM")
print(f"O2 depletion:         {(O2_depletion/inlet_O2)*100:.1f}%")
print()
print(f"Total glucose consump: {glucose_consumption:.3f} μmol/min")
print(f"Inlet glucose:        {inlet_glucose:.1f} μM ({inlet_glucose/1000:.1f} mM)")
print(f"Outlet glucose:       {outlet_glucose:.1f} μM ({outlet_glucose/1000:.1f} mM)")
print(f"Glucose depletion:    {(glucose_depletion/inlet_glucose)*100:.1f}%")

# Viability assessment
print(f"\n✓ VIABILITY ASSESSMENT:")
print("-" * 50)

viable = True

if outlet_O2 >= 40:  # 20% of atmospheric
    print(f"✅ Oxygen adequate ({outlet_O2:.1f} μM > 40 μM threshold)")
else:
    print(f"❌ Oxygen insufficient ({outlet_O2:.1f} μM < 40 μM threshold)")
    print(f"   Solution: Increase flow rate to >{ (O2_consumption/(inlet_O2-40))*1e6:.1f} μL/min")
    viable = False

if outlet_glucose >= 1000:  # 1 mM
    print(f"✅ Glucose adequate ({outlet_glucose/1000:.1f} mM > 1 mM threshold)")
else:
    print(f"❌ Glucose insufficient ({outlet_glucose/1000:.1f} mM < 1 mM threshold)")
    print(f"   Solution: Increase flow rate or inlet glucose concentration")
    viable = False

# ============================================================================
# SECTION 4: SHEAR STRESS ANALYSIS
# ============================================================================

print("\n" + "="*80)
print("SECTION 4: SHEAR STRESS EVALUATION")
print("="*80)

def calculate_shear_stress_perfusion(flow_rate_uL, width_cm, height_cm, viscosity=0.001):
    """Calculate wall shear stress in perfusion channel"""
    # Convert flow rate to m³/s
    Q = flow_rate_uL * 1e-9 / 60

    # Convert dimensions to meters
    w = width_cm / 100
    h = height_cm / 100

    # For rectangular channel
    tau = (6 * viscosity * Q) / (w * h**2)
    return tau

# Assume square cross-section for simplicity
channel_width = np.sqrt(chip_area)  # cm (approximate)
shear_stress = calculate_shear_stress_perfusion(flow_rate, channel_width, channel_height)

print(f"\n💨 SHEAR STRESS ANALYSIS:")
print("-" * 50)
print(f"Channel width:        {channel_width:.2f} cm")
print(f"Channel height:       {channel_height*1e4:.0f} μm")
print(f"Calculated shear:     {shear_stress:.3f} Pa")
print(f"Cell tolerance:       {cell_props['shear_tolerance']} Pa")

if shear_stress <= cell_props['shear_tolerance']:
    print(f"✅ Shear stress acceptable ({shear_stress:.3f} Pa ≤ {cell_props['shear_tolerance']} Pa)")
else:
    print(f"❌ Shear stress too high ({shear_stress:.3f} Pa > {cell_props['shear_tolerance']} Pa)")
    max_safe_flow = (cell_props['shear_tolerance'] * channel_width * (channel_height**2)) / (6 * 0.001) * 60 * 1e9
    print(f"   Solution: Reduce flow rate to <{max_safe_flow:.1f} μL/min or increase channel height")
    viable = False

# ============================================================================
# SECTION 5: OPTIMIZATION RECOMMENDATIONS
# ============================================================================

print("\n" + "="*80)
print("SECTION 5: SYSTEM OPTIMIZATION")
print("="*80)

if viable:
    print("\n🎉 CURRENT DESIGN IS VIABLE!")
    print("✓ Adequate oxygen delivery")
    print("✓ Adequate glucose supply")
    print("✓ Acceptable shear stress")
else:
    print("\n⚠️ DESIGN OPTIMIZATION NEEDED")
    print("\nOptimization strategies:")

    # Calculate optimal flow rate
    # Constraint 1: Oxygen (outlet should be >40 μM)
    min_flow_O2 = (O2_consumption / (inlet_O2 - 40)) * 1e6

    # Constraint 2: Glucose (outlet should be >1000 μM)
    min_flow_glucose = (glucose_consumption / (inlet_glucose - 1000)) * 1e6

    # Constraint 3: Shear stress (should be < tolerance)
    max_flow_shear = (cell_props['shear_tolerance'] * channel_width * (channel_height**2)) / (6 * 0.001) * 60 * 1e9

    print(f"\n📊 Flow rate constraints:")
    print(f"  Minimum for O2:     {min_flow_O2:.1f} μL/min")
    print(f"  Minimum for glucose: {min_flow_glucose:.1f} μL/min")
    print(f"  Maximum for shear:  {max_flow_shear:.1f} μL/min")

    if min_flow_O2 < max_flow_shear and min_flow_glucose < max_flow_shear:
        optimal_flow = max(min_flow_O2, min_flow_glucose) * 1.2  # 20% safety margin
        print(f"\n✅ Optimal flow rate: {optimal_flow:.1f} μL/min")
    else:
        print(f"\n❌ No viable flow rate exists!")
        print(f"   Solution options:")
        print(f"   1. Reduce cell density")
        print(f"   2. Increase channel height (reduces shear)")
        print(f"   3. Increase chip area (reduces cell density per volume)")

# ============================================================================
# SECTION 6: VISUALIZATION
# ============================================================================

print("\n" + "="*80)
print("SECTION 6: PERFUSION SYSTEM VISUALIZATION")
print("="*80)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Flow rate vs outlet concentrations
ax1 = axes[0, 0]
flow_range = np.linspace(1, 500, 100)
O2_outlets = []
glucose_outlets = []

for fr in flow_range:
    O2_out, _ = calculate_oxygen_depletion(O2_consumption, fr, inlet_O2)
    glucose_out = inlet_glucose - (glucose_consumption / (fr * 1e-6))
    O2_outlets.append(O2_out)
    glucose_outlets.append(glucose_out)

ax1_twin = ax1.twinx()
line1 = ax1.plot(flow_range, O2_outlets, 'b-', linewidth=2, label='Oxygen')
line2 = ax1_twin.plot(flow_range, np.array(glucose_outlets)/1000, 'r-', linewidth=2, label='Glucose')

ax1.axhline(40, color='blue', linestyle='--', linewidth=2, alpha=0.5, label='O2 threshold')
ax1_twin.axhline(1, color='red', linestyle='--', linewidth=2, alpha=0.5, label='Glucose threshold')
ax1.axvline(flow_rate, color='green', linestyle=':', linewidth=2, label='Current flow')

ax1.set_xlabel('Flow Rate (μL/min)', fontsize=11, fontweight='bold')
ax1.set_ylabel('Outlet O2 (μM)', fontsize=11, fontweight='bold', color='blue')
ax1_twin.set_ylabel('Outlet Glucose (mM)', fontsize=11, fontweight='bold', color='red')
ax1.set_title(f'Nutrient Delivery vs Flow Rate\n({cell_type})', fontsize=12, fontweight='bold')
ax1.tick_params(axis='y', labelcolor='blue')
ax1_twin.tick_params(axis='y', labelcolor='red')
ax1.grid(True, alpha=0.3)

# Plot 2: Cell density vs minimum flow rate
ax2 = axes[0, 1]
densities = np.logspace(4, 6, 50)
min_flows = []

for dens in densities:
    cells = chip_area * dens
    O2_cons = calculate_oxygen_consumption(cells, cell_props['OCR'])
    min_fr = (O2_cons / (inlet_O2 - 40)) * 1e6
    min_flows.append(min_fr)

ax2.plot(densities, min_flows, 'g-', linewidth=2)
ax2.axhline(flow_rate, color='red', linestyle='--', linewidth=2, label='Current flow rate')
ax2.axvline(cell_density, color='blue', linestyle=':', linewidth=2, label='Current density')
ax2.set_xlabel('Cell Density (cells/cm²)', fontsize=11, fontweight='bold')
ax2.set_ylabel('Minimum Flow Rate (μL/min)', fontsize=11, fontweight='bold')
ax2.set_title('Cell Density Impact on Flow Requirements', fontsize=12, fontweight='bold')
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.legend()
ax2.grid(True, alpha=0.3, which='both')

# Plot 3: Shear stress vs flow rate
ax3 = axes[1, 0]
shear_values = []

for fr in flow_range:
    shear = calculate_shear_stress_perfusion(fr, channel_width, channel_height)
    shear_values.append(shear)

ax3.plot(flow_range, shear_values, 'purple', linewidth=2)
ax3.axhline(cell_props['shear_tolerance'], color='red', linestyle='--', linewidth=2,
           label='Cell tolerance limit')
ax3.axvline(flow_rate, color='green', linestyle=':', linewidth=2, label='Current flow')
ax3.fill_between(flow_range, 0, cell_props['shear_tolerance'], alpha=0.2, color='green',
                label='Safe zone')

ax3.set_xlabel('Flow Rate (μL/min)', fontsize=11, fontweight='bold')
ax3.set_ylabel('Shear Stress (Pa)', fontsize=11, fontweight='bold')
ax3.set_title('Shear Stress vs Flow Rate', fontsize=12, fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Comparison across cell types
ax4 = axes[1, 1]
cell_types = list(cell_metabolic_data.keys())
optimal_flows = []
shear_limits = []

for ct in cell_types:
    props = cell_metabolic_data[ct]
    cells = chip_area * props['density']
    O2_cons = calculate_oxygen_consumption(cells, props['OCR'])
    min_fr = (O2_cons / (inlet_O2 - 40)) * 1e6
    optimal_flows.append(min_fr)
    shear_limits.append(props['shear_tolerance'])

x = np.arange(len(cell_types))
width_bar = 0.35

bars1 = ax4.bar(x - width_bar/2, optimal_flows, width_bar, label='Min flow (O2)', alpha=0.7)
bars2 = ax4.bar(x + width_bar/2, [sl*50 for sl in shear_limits], width_bar,
               label='Shear tolerance (×50)', alpha=0.7)

ax4.set_xlabel('Cell Type', fontsize=11, fontweight='bold')
ax4.set_ylabel('Flow Rate (μL/min) / Shear (Pa×50)', fontsize=11, fontweight='bold')
ax4.set_title('Perfusion Requirements by Cell Type', fontsize=12, fontweight='bold')
ax4.set_xticks(x)
ax4.set_xticklabels(cell_types, rotation=45, ha='right')
ax4.legend()
ax4.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# ============================================================================
# SECTION 7: STUDENT EXPLORATION TASKS
# ============================================================================

print("\n" + "="*80)
print("SECTION 7: HANDS-ON EXPLORATION")
print("="*80)

print("""
🎯 STUDENT TASKS:

Task 1: OPTIMIZE FOR LONG-TERM CULTURE
   Goal: Maintain hepatocytes viable for 7 days
   • What is the optimal flow rate?
   • How much medium is consumed per day?
   • Design a reservoir and pumping system

Task 2: MULTI-ORGAN SYSTEM
   Connect liver chip (hepatocytes) → kidney chip (kidney cells)
   • Liver outlet feeds kidney inlet
   • Calculate cascade effects on O2 and glucose
   • Is the kidney chip still viable?

Task 3: SCALE-UP ANALYSIS
   You want 10x more cells for higher signal
   • Keep same chip area, increase density 10×
   • OR: Keep density, increase area 10×
   • Which approach works better? Why?

Task 4: PULSATILE FLOW
   Simulate heartbeat: 1 Hz pulsation, ±20% flow variation
   • Calculate peak and minimum shear stress
   • Are cells still safe during peak flow?
   • What about oxygen during minimum flow?

Task 5: DESIGN A LIVER-CHIP
   Requirements:
   • 1 million hepatocytes
   • 7-day culture
   • Physiologically relevant shear (<0.1 Pa)
   • Determine: chip dimensions, flow rate, reservoir size

MODIFY THESE PARAMETERS IN SECTION 2 AND RE-RUN:
   cell_type = 'Hepatocytes'  # Try all cell types
   chip_area = 1.0            # Try: 0.1-10 cm²
   channel_height = 100e-4    # Try: 50e-4 to 500e-4 cm
   flow_rate = 50             # Try: 1-500 μL/min
""")

# ============================================================================
# SECTION 8: REFLECTION QUESTIONS
# ============================================================================

print("\n" + "="*80)
print("SECTION 8: REFLECTION QUESTIONS")
print("="*80)

print("""
Answer these questions based on your exploration:

1. STATIC VS PERFUSED CULTURE:
   Compare the advantages of perfusion over static culture
   In what scenarios is static culture still acceptable?

2. METABOLIC DIFFERENCES:
   Why do cardiomyocytes require higher flow rates than endothelial cells?
   How does this affect multi-organ chip design?

3. SHEAR STRESS BALANCE:
   Some cells need physiological shear stress for proper function (endothelial)
   How do you balance this with metabolic oxygen demands?

4. SENSOR INTEGRATION:
   Where would you place O2 and glucose sensors in the system?
   Inlet, outlet, or both? Why?

5. REAL-WORLD CHALLENGES:
   What happens during pump failure overnight?
   How would you design fail-safes for long-term experiments?

Write your answers here:
__________________________________________________________________________
__________________________________________________________________________
__________________________________________________________________________
""")

print("\n" + "="*80)
print("🎉 EXERCISE 4 COMPLETE!")
print("="*80)
print("Key Takeaways:")
print("✓ Perfusion balances nutrient delivery with shear stress")
print("✓ Cell metabolic rates determine minimum flow requirements")
print("✓ Different cell types have vastly different requirements")
print("✓ Multi-organ systems require careful cascade design")
print("\n📚 Ready for Exercise 5: Multi-Organ System Integration!")